In [1]:
from __future__ import annotations

from typing import Union

import numpy as np
import pandas as pd

In [2]:
players_2324_path = "./Quotazioni_Fantacalcio_Stagione_2023_24.csv"
players_2223_path = "./Quotazioni_Fantacalcio_Stagione_2022_23.csv"
stats_2223_path = "./Statistiche_Fantacalcio_Stagione_2022_23.csv"

players_2324_df = pd.read_csv(players_2324_path, delimiter=",", header=1, index_col=False)
players_2223_df = pd.read_csv(players_2223_path, delimiter=",", header=1, index_col=False)
stats_2223_df = pd.read_csv(stats_2223_path, delimiter=",", header=1, index_col=False)

# basic functions

In [23]:
class Filter:

    def __init__(
            self: Filter, column: str, operation: str, value: Union[str, float]) -> None:
        self.column = column
        self.operation = operation
        self.value = value
    
    def apply_to_df(self: Filter, df: pd.DataFrame) -> pd.DataFrame:
        if self.operation == "ge":
            return df[df[self.column] > self.value]
        elif self.operation == "geq":
            return df[df[self.column] >= self.value]
        elif self.operation == "le":
            return df[df[self.column] < self.value]
        elif self.operation == "leq":
            return df[df[self.column] <= self.value]
        elif self.operation == "eq":
            return df[df[self.column] == self.value]
        elif self.operation == "neq":
            return df[df[self.column] != self.value]


def apply_multiple_filters(df: pd.DataFrame, filter_list: list[Filter]) -> pd.DataFrame:
    filtered_df = df.copy()
    for filter in filter_list:
        filtered_df = filter.apply_to_df(filtered_df)
    return filtered_df


def extract_players_mantra_position(df: pd.DataFrame, position: str) -> pd.DataFrame:
    """
    The mantra position is tricky, because they actually include multiple positions with ; as delimiter.
    The easiest way is to first filter the players that have also that specific position.
    """
    return df[df["RM"].str.contains(position)]


In [21]:
# example on how to use a filter
filter = Filter("Qt.A M", "geq", 15)
good_players = filter.apply_to_df(players_2324_df)
good_players

,Id,R,RM,Nome,Squadra,Qt.A,Qt.I,Diff.,Qt.A M,Qt.I M,Diff.M,FVM,FVM M
0,572,P,Por,Meret,Napoli,17,17,0,17,17,0,80,80
1,4312,P,Por,Maignan,Milan,17,17,0,17,17,0,89,89
2,2814,P,Por,Provedel,Lazio,16,16,0,16,16,0,68,68
3,453,P,Por,Szczesny,Juventus,15,15,0,15,15,0,71,71
4,2428,P,Por,Sommer,Inter,15,15,0,15,15,0,67,67
...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,6409,A,Pc,Beltran L.,Fiorentina,16,16,0,16,16,0,82,78
457,1939,A,Pc,Lapadula,Cagliari,15,15,0,15,15,0,41,35
458,2544,A,Pc,Abraham,Roma,15,15,0,15,15,0,58,50
459,2819,A,Pc,Caputo,Empoli,15,15,0,15,15,0,73,60


In [22]:
# example on how to use multiple filters to extract cheat but good goalkeepers
filter_list = [
    Filter("RM", "eq", "Por"),
    Filter("Qt.A M", "geq", 10),
    Filter("FVM M", "leq", 50)
]
filtered_df = apply_multiple_filters(players_2324_df, filter_list)
filtered_df

,Id,R,RM,Nome,Squadra,Qt.A,Qt.I,Diff.,Qt.A M,Qt.I M,Diff.M,FVM,FVM M
6,5876,P,Por,Di Gregorio,Monza,12,12,0,12,12,0,37,37
7,4431,P,Por,Carnesecchi,Atalanta,11,11,0,11,11,0,28,28
8,133,P,Por,Skorupski,Bologna,10,10,0,10,10,0,32,32
9,2170,P,Por,Milinkovic-Savic V.,Torino,10,10,0,10,10,0,31,31
10,2815,P,Por,Terracciano,Fiorentina,10,10,0,10,10,0,31,31


In [24]:
# example: players who can play also as difensori centrali
difensori_centrali = extract_players_mantra_position(players_2324_df, "Dc")
difensori_centrali

,Id,R,RM,Nome,Squadra,Qt.A,Qt.I,Diff.,Qt.A M,Qt.I M,Diff.M,FVM,FVM M
65,6066,D,Dd;Dc,Posch,Bologna,16,16,0,18,18,0,53,56
67,4237,D,Dd;Ds;Dc,Danilo,Juventus,15,15,0,18,18,0,44,58
68,4245,D,Dc,Smalling,Roma,15,15,0,15,15,0,55,55
69,4409,D,Dc,Rrahmani,Napoli,14,14,0,14,14,0,44,44
70,6041,D,Dc,Schuurs,Torino,14,14,0,14,14,0,46,46
...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,6244,D,Dc,Matturro,Genoa,1,1,0,1,1,0,1,1
239,6253,D,Dc,N'guessan,Torino,1,1,0,1,1,0,1,1
243,6411,D,Dc,Mateus Lusuardi,Frosinone,1,1,0,1,1,0,1,1
244,6430,D,Dc,Kalaj,Frosinone,1,1,0,1,1,0,1,1


# extracting interesting players: Andrea

In [32]:
names = ["Zapata D.", "Scalvini", "Toure' E."]

players_2324_df[np.isin(players_2324_df["Nome"], names)]

,Id,R,RM,Nome,Squadra,Qt.A,Qt.I,Diff.,Qt.A M,Qt.I M,Diff.M,FVM,FVM M
99,5526,D,Dc,Scalvini,Atalanta,9,9,0,9,9,0,30,30
451,608,A,Pc,Zapata D.,Atalanta,19,19,0,19,19,0,85,85
455,6229,A,Pc,Toure' E.,Atalanta,16,16,0,15,15,0,50,45


In [30]:
Filter("Squadra", "eq", "Atalanta").apply_to_df(players_2324_df)

,Id,R,RM,Nome,Squadra,Qt.A,Qt.I,Diff.,Qt.A M,Qt.I M,Diff.M,FVM,FVM M
7,4431,P,Por,Carnesecchi,Atalanta,11,11,0,11,11,0,28,28
21,2792,P,Por,Musso,Atalanta,3,3,0,3,3,0,17,17
42,2297,P,Por,Rossi F.,Atalanta,1,1,0,1,1,0,1,1
66,554,D,E,Zappacosta,Atalanta,15,15,0,14,14,0,48,42
75,5067,D,E,Bakker,Atalanta,12,12,0,11,11,0,42,35
93,695,D,Dc,Toloi,Atalanta,9,9,0,9,9,0,19,19
95,2640,D,Ds;Dc,Kolasinac,Atalanta,9,9,0,10,10,0,20,23
99,5526,D,Dc,Scalvini,Atalanta,9,9,0,9,9,0,30,30
109,4332,D,Dc,Demiral,Atalanta,8,8,0,8,8,0,11,11
116,787,D,Dc,Djimsiti,Atalanta,7,7,0,7,7,0,12,12
